In [2]:
from aocd import get_data
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

day9 = get_data(day=9,year=2021)

Trying this with a moving window

In [3]:
heightmap = np.array([[int(j) for j in i] for i in day9.split('\n')])
heightmap = np.pad(heightmap,1,'constant',constant_values=10)


lowvals = np.minimum(sliding_window_view(heightmap,(3,1)).min(axis=-1).min(axis=-1)[:,1:101],sliding_window_view(heightmap,(1,3)).min(axis=-1).min(axis=-1)[1:101,:])

print(np.sum(np.where(heightmap[1:101,1:101]==lowvals,lowvals+1,0)))

1820


That's the wrong answer! It includes plateaus where a point is equal to its neighbours, not less than.

In [4]:
lowvals = np.zeros((102,102))-1
heightmap = np.array([[int(j) for j in i] for i in day9.split('\n')])
heightmap = np.pad(heightmap,1,'constant',constant_values=10)
for i in range(1,101):
    for j in range(1,101):
        if (heightmap[i,j] < heightmap[i-1,j]) & (heightmap[i,j] < heightmap[i+1,j]) & (heightmap[i,j] < heightmap[i,j-1]) & (heightmap[i,j] < heightmap[i,j+1]):
            lowvals[i,j] = heightmap[i,j]
            
print(np.sum(np.where(heightmap==lowvals,lowvals+1,0)))


530.0


That's correct! On to Part II.

In [5]:
import pandas as pd
basins = lowvals.copy()
b = 0

for i in range(1,101):
    for j in range(1,101):
        if lowvals[i,j] >= 0:
            basins[i,j] = b
            b += 1

changes = 1
while changes > 0:
    changes = 0
    for i in range(1,101):
        for j in range(1,101):
            if basins[i,j] >= 0:
                if (heightmap[i-1,j] < 9) & (basins[i-1,j] < 0):
                    changes += 1
                    basins[i-1,j] = basins[i,j]
                if (heightmap[i+1,j] < 9) & (basins[i+1,j] < 0):
                    changes += 1
                    basins[i+1,j] = basins[i,j]
                if (heightmap[i,j-1] < 9) & (basins[i,j-1] < 0):
                    changes += 1
                    basins[i,j-1] = basins[i,j]
                if (heightmap[i,j+1] < 9) & (basins[i,j+1] < 0):
                    changes += 1
                    basins[i,j+1] = basins[i,j]
    print(changes)

print(pd.DataFrame(basins.flatten()).replace(-1,np.nan).value_counts()[0:3].product())


3486
1191
904
654
434
241
117
38
0
1019494


Got it!